# 事前準備

## Google Drive

Google Driveをマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 地域コード

GoogleDriveから地域一覧を取得する

In [ ]:
import json

with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)

print(arealist)

# 関数テスト

estat-APIのパラメータ作成

In [41]:
params = {
    'statsDataId':'0000020201',
    'cdCat01':'A1101',
    'cdArea': '28100,28101,28102,28105,28106,28107,28108,28109,28110,28111,28201,28202,28203,28204,28205,28206,28207,28208,28209,28210,28212,28213,28214,28215,28216,28217,28218,28219,28220,28221,28222,28223,28224,28225,28226,28227,28228,28229,28301,28365,28381,28382,28442,28443,28446,28464,28481,28501,28585,28586'
    }

## estat-APIのレスポンスを取得

estat-APIのレスポンス（JSON）を取得する関数

In [7]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

import urllib.parse
import urllib.request
import json

"""
estat-APIのレスポンスを取得する関数
"""

def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [ ]:
res = get_estat_response(params)
print(res)

## JSONをDataFrameに変換

estat-APIのレスポンスをDataFrameに変換する

In [16]:
import pandas as pd

"""
estat-APIのレスポンスをDataFrameに変換する
"""

def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

In [ ]:
df1 = conv_estat_response_to_dataframe(res)
df1

## DataFrameを整形

In [42]:
"""
DataFrameを整形する
"""

def  format_estat_dataframe(df_arg):
  # 必要な列だけ抽出
  df_res =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','$':'value','@unit':'unit'}
  df_res = df_res.rename(columns=columns)

  # 欠損データ削除
  df_res = df_res.dropna()

  # '-'を'0'に置換
  df_res = df_res.replace('-', '0')

  # categoryNameから不要な情報（categoryCode）を削除
  df_res['categoryName'] = df_res.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df_res['timeCode'] = df_res.apply(lambda x: x['timeCode'][:4], 1)
  
  # 地域情報をマージ
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    df_arealist = pd.json_normalize(arealist)  
  df_res = pd.merge(df_res, df_arealist, on='areaCode', how='left')

  return df_res

In [52]:
df2 = format_estat_dataframe(df1)
df2

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
0,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28100,1367390,人,1296,28,兵庫県,28100,神戸市,2,神戸市,city
1,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28101,183284,人,1297,28,兵庫県,28101,神戸市東灘区,1,神戸市東灘区,city
2,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28102,142313,人,1298,28,兵庫県,28102,神戸市灘区,1,神戸市灘区,city
3,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28105,142418,人,1299,28,兵庫県,28105,神戸市兵庫区,1,神戸市兵庫区,city
4,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28106,163949,人,1300,28,兵庫県,28106,神戸市長田区,1,神戸市長田区,city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,28464,33690,人,1341,28,兵庫県,28464,太子町,0,太子町,city
395,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,28481,15224,人,1342,28,兵庫県,28481,上郡町,0,上郡町,city
396,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,28501,17510,人,1343,28,兵庫県,28501,佐用町,0,佐用町,city
397,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,28585,18070,人,1344,28,兵庫県,28585,香美町,0,香美町,city


# まとめ

結果をまとめる

In [51]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

import urllib.parse
import urllib.request
import json
import pandas as pd

"""
estat-APIのレスポンスを整形してDataFrameに変換
"""

def get_estat_dataframe(params):
  res = get_estat_response(params)
  df_res = conv_estat_response_to_dataframe(res)
  df_res = format_estat_dataframe(df_res)

  return df_res

"""
estat-APIのレスポンスを取得する関数
"""

def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))


"""
estat-APIのレスポンスをDataFrameに変換する
"""

def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

"""
DataFrameを整形する
"""

def  format_estat_dataframe(df_arg):
  # 必要な列だけ抽出
  df_res =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','$':'value','@unit':'unit'}
  df_res = df_res.rename(columns=columns)

  # 欠損データ削除
  df_res = df_res.dropna()

  # '-'を'0'に置換
  df_res = df_res.replace('-', '0')

  # categoryNameから不要な情報（categoryCode）を削除
  df_res['categoryName'] = df_res.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df_res['timeCode'] = df_res.apply(lambda x: x['timeCode'][:4], 1)
  
  # 地域情報をマージ
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    df_arealist = pd.json_normalize(arealist)  
  df_res = pd.merge(df_res, df_arealist, on='areaCode', how='left')

  return df_res

# テスト

## 市区町村の総人口

In [45]:
params = {
    'statsDataId':'0000020201',
    'cdCat01':'A1101',
    'cdArea': '28100,28101,28102,28105,28106,28107,28108,28109,28110,28111,28201,28202,28203,28204,28205,28206,28207,28208,28209,28210,28212,28213,28214,28215,28216,28217,28218,28219,28220,28221,28222,28223,28224,28225,28226,28227,28228,28229,28301,28365,28381,28382,28442,28443,28446,28464,28481,28501,28585,28586'
    }

In [47]:
df_population_city = get_estat_dataframe(params)
df_population_city

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
0,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28100,1367390,人,1296,28,兵庫県,28100,神戸市,2,神戸市,city
1,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28101,183284,人,1297,28,兵庫県,28101,神戸市東灘区,1,神戸市東灘区,city
2,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28102,142313,人,1298,28,兵庫県,28102,神戸市灘区,1,神戸市灘区,city
3,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28105,142418,人,1299,28,兵庫県,28105,神戸市兵庫区,1,神戸市兵庫区,city
4,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,28106,163949,人,1300,28,兵庫県,28106,神戸市長田区,1,神戸市長田区,city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,28464,33690,人,1341,28,兵庫県,28464,太子町,0,太子町,city
395,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,28481,15224,人,1342,28,兵庫県,28481,上郡町,0,上郡町,city
396,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,28501,17510,人,1343,28,兵庫県,28501,佐用町,0,佐用町,city
397,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,28585,18070,人,1344,28,兵庫県,28585,香美町,0,香美町,city


## 都道府県の総人口

In [48]:
params = {
    'statsDataId':'0000010101',
    'cdCat01':'A1101',
    'cdArea': '01000,02000,03000,04000,05000,06000,07000,08000,09000,10000,11000,12000,13000,14000,15000,16000,17000,18000,19000,20000,21000,22000,23000,24000,25000,26000,27000,28000,29000,30000,31000,32000,33000,34000,35000,36000,37000,38000,39000,40000,41000,42000,43000,44000,45000,46000,47000'
    }

In [49]:
df_population_prefecture = get_estat_dataframe(params)
df_population_prefecture

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
0,0000010101,社会・人口統計体系,A1101,総人口,1975,1975年度,01000,5338206,人,0,1,北海道,None,None,None,北海道,prefecture
1,0000010101,社会・人口統計体系,A1101,総人口,1975,1975年度,02000,1468646,人,1,2,青森県,None,None,None,青森県,prefecture
2,0000010101,社会・人口統計体系,A1101,総人口,1975,1975年度,03000,1385563,人,2,3,岩手県,None,None,None,岩手県,prefecture
3,0000010101,社会・人口統計体系,A1101,総人口,1975,1975年度,04000,1955267,人,3,4,宮城県,None,None,None,宮城県,prefecture
4,0000010101,社会・人口統計体系,A1101,総人口,1975,1975年度,05000,1232481,人,4,5,秋田県,None,None,None,秋田県,prefecture
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110,0000010101,社会・人口統計体系,A1101,総人口,2019,2019年度,43000,1748000,人,42,43,熊本県,None,None,None,熊本県,prefecture
2111,0000010101,社会・人口統計体系,A1101,総人口,2019,2019年度,44000,1135000,人,43,44,大分県,None,None,None,大分県,prefecture
2112,0000010101,社会・人口統計体系,A1101,総人口,2019,2019年度,45000,1073000,人,44,45,宮崎県,None,None,None,宮崎県,prefecture
2113,0000010101,社会・人口統計体系,A1101,総人口,2019,2019年度,46000,1602000,人,45,46,鹿児島県,None,None,None,鹿児島県,prefecture
